- Reference:
  - https://tech.enigmo.co.jp/entry/2018/12/10/170959
  - https://github.com/optuna/optuna/blob/master/examples/lightgbm_simple.py

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

In [2]:
DATA_DIR_PATH = os.path.join('..', '..', 'data')

def laod_data(file_name):
  file_path = os.path.join(DATA_DIR_PATH, file_name)
  return pd.read_csv(file_path, header=0)

In [3]:
df_train = laod_data('bookmark_train_data.csv')
print(len(df_train))
#df_train.head(3)

199


In [4]:
from sklearn.preprocessing import LabelEncoder

cat_le = LabelEncoder()
cat_le.fit(df_train['category'])

WORD_CLASSE_LABELS = ['その他', 'フィラー', '副詞', '助動詞', '助詞',
                     '動詞', '名詞', '形容詞', '感動詞', '接続詞',
                     '接頭詞', '記号', '連体詞',]
new_wc_columns = {}
for (i, name) in enumerate(WORD_CLASSE_LABELS):
    new_wc_columns['wc_sum_{}'.format(name)] = 'wc_sum_{}'.format(i)
    new_wc_columns['wc_rate_{}'.format(name)] = 'wc_rate_{}'.format(i)


def data_preprocess(df):
    new_df = df.drop(['ncode', 'title', 'bookmark_cat'], axis=1)

    new_df['category'] = cat_le.transform(new_df['category'])

    new_df['rating'] = new_df['rating'] * 10
    new_df = new_df.astype({'rating': 'int32'})

    #wc_columns = [col for col in new_df.columns if col.find('wc_') != -1]
    #new_df = new_df.drop(wc_columns, axis=1)
    new_df = new_df.rename(columns=new_wc_columns)

    kw_columns = [col for col in new_df.columns if col.find('kw_') != -1]
    new_df = new_df.drop(kw_columns, axis=1)

    return new_df

In [5]:
df_train.keys()

Index(['ncode', 'title', 'category', 'bookmark_cat', 'created_at',
       'updated_at', 'char_count_sum', 'new_line_count_sum',
       'talk_char_count_sum', 'new_line_count_rate_sum',
       'talk_char_count_rate_sum', 'char_count_avg', 'new_line_count_avg',
       'talk_char_count_avg', 'new_line_count_rate_avg',
       'talk_char_count_rate_avg', 'wc_sum_その他', 'wc_sum_フィラー', 'wc_sum_副詞',
       'wc_sum_助動詞', 'wc_sum_助詞', 'wc_sum_動詞', 'wc_sum_名詞', 'wc_sum_形容詞',
       'wc_sum_感動詞', 'wc_sum_接続詞', 'wc_sum_接頭詞', 'wc_sum_記号', 'wc_sum_連体詞',
       'wc_rate_その他', 'wc_rate_フィラー', 'wc_rate_副詞', 'wc_rate_助動詞',
       'wc_rate_助詞', 'wc_rate_動詞', 'wc_rate_名詞', 'wc_rate_形容詞', 'wc_rate_感動詞',
       'wc_rate_接続詞', 'wc_rate_接頭詞', 'wc_rate_記号', 'wc_rate_連体詞', 'kw_冒険者',
       'kw_魔王', 'kw_日常', 'kw_成り上がり', 'kw_内政', 'kw_恋愛', 'kw_無双', 'kw_青春',
       'kw_ハーレム', 'kw_男主人公', 'kw_奴隷', 'kw_テンプレ', 'kw_勘違い', 'kw_ほのぼの', 'kw_貴族',
       'kw_ギルド', 'kw_コメディ', 'kw_異世界', 'kw_エルフ', 'kw_ファンタジー', 'kw_オリジナル戦記',
       

In [6]:
df_train2 = data_preprocess(df_train)

In [7]:
df_train2.keys()

Index(['category', 'created_at', 'updated_at', 'char_count_sum',
       'new_line_count_sum', 'talk_char_count_sum', 'new_line_count_rate_sum',
       'talk_char_count_rate_sum', 'char_count_avg', 'new_line_count_avg',
       'talk_char_count_avg', 'new_line_count_rate_avg',
       'talk_char_count_rate_avg', 'wc_sum_0', 'wc_sum_1', 'wc_sum_2',
       'wc_sum_3', 'wc_sum_4', 'wc_sum_5', 'wc_sum_6', 'wc_sum_7', 'wc_sum_8',
       'wc_sum_9', 'wc_sum_10', 'wc_sum_11', 'wc_sum_12', 'wc_rate_0',
       'wc_rate_1', 'wc_rate_2', 'wc_rate_3', 'wc_rate_4', 'wc_rate_5',
       'wc_rate_6', 'wc_rate_7', 'wc_rate_8', 'wc_rate_9', 'wc_rate_10',
       'wc_rate_11', 'wc_rate_12', 'rating'],
      dtype='object')

In [8]:
df_train2.head(3)

,category,created_at,updated_at,char_count_sum,new_line_count_sum,talk_char_count_sum,new_line_count_rate_sum,talk_char_count_rate_sum,char_count_avg,new_line_count_avg,...,wc_rate_4,wc_rate_5,wc_rate_6,wc_rate_7,wc_rate_8,wc_rate_9,wc_rate_10,wc_rate_11,wc_rate_12,rating
0,6,1.586338e+09,1.593468e+09,92340,3840,28950,0.041585,0.313515,3078.0,128.0,...,0.237664,0.226083,0.225076,0.013595,0.006042,0.007049,0.006042,0.144008,0.011078,30
1,6,1.590714e+09,1.593338e+09,77880,4140,4950,0.053159,0.063559,2596.0,138.0,...,0.238590,0.238013,0.217215,0.017909,0.001155,0.010399,0.001155,0.138070,0.013287,40
2,6,1.575176e+09,1.593429e+09,59880,3090,10230,0.051603,0.170842,1996.0,103.0,...,0.240486,0.177328,0.285020,0.014575,0.000000,0.005668,0.008907,0.148178,0.005668,30


In [9]:
X = df_train2.drop(['rating'], axis=1).values
t = df_train2['rating'].values
print(X.shape, t.shape)

(199, 39) (199,)


In [10]:
feature_names = df_train2.keys()[:-1]

In [11]:
from imblearn.over_sampling import RandomOverSampler

In [12]:
ros = RandomOverSampler(random_state=0)
X_resampled, t_resampled = ros.fit_resample(X, t)

print(X_resampled.shape, t_resampled.shape)

u, counts = np.unique(t_resampled, return_counts=True)

print(len(u))
print(u)
print(counts)

(915, 39) (915,)
15
[ 5 10 15 17 20 25 30 35 40 45 50 52 60 67 75]
[61 61 61 61 61 61 61 61 61 61 61 61 61 61 61]


In [13]:
from sklearn.model_selection import train_test_split

#X_train, X_val, t_train, t_val = train_test_split(X, t, test_size=0.3, shuffle=True, random_state=1)
X_train, X_val, t_train, t_val = train_test_split(X_resampled, t_resampled, test_size=0.3, shuffle=True, random_state=1)

In [14]:
print(X_train.shape, t_train.shape)
print(X_val.shape, t_val.shape)

(640, 39) (640,)
(275, 39) (275,)


In [15]:
import lightgbm as lgb

In [16]:
import optuna

In [21]:
from sklearn.metrics import mean_squared_error

In [22]:
def objective(trial):
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    #train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    #dtrain = lgb.Dataset(train_x, label=train_y)
    lgb_train = lgb.Dataset(X_train, t_train)
    lgb_eval = lgb.Dataset(X_val, t_val, reference=lgb_train)

    param = {
        #'objective': 'binary',
        #'metric': 'binary_logloss',
        'objective': 'mean_squared_error',
        'metric': 'rmse',
        'verbosity': -1,
        #'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    #evals_result = {}
    #gbm = lgb.train(params,
    #                lgb_train,
    #                valid_sets=lgb_eval,
    #                num_boost_round=2000,
    #                early_stopping_rounds=20,
    #                feature_name=feature_names.to_list(),
    #                evals_result=evals_result,
    #                verbose_eval=10)

    gbm = lgb.train(param, lgb_train)
    preds = gbm.predict(X_val)
    mse = mean_squared_error(t_val, preds)
    rmse = math.sqrt(mse)

    return rmse

In [24]:
#study = optuna.create_study(direction='maximize')
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2020-09-11 23:48:55,826] Trial 0 finished with value: 4.626589429747335 and parameters: {'lambda_l1': 0.1339522018079735, 'lambda_l2': 0.44935902640140873, 'num_leaves': 169, 'feature_fraction': 0.44027562443822993, 'bagging_fraction': 0.9003190764575784, 'bagging_freq': 5, 'min_child_samples': 11}. Best is trial 0 with value: 4.626589429747335.
[I 2020-09-11 23:48:55,880] Trial 1 finished with value: 6.482503120050631 and parameters: {'lambda_l1': 0.0006053607734179018, 'lambda_l2': 0.2261284596046648, 'num_leaves': 61, 'feature_fraction': 0.7899458351692836, 'bagging_fraction': 0.7993137160036858, 'bagging_freq': 2, 'min_child_samples': 31}. Best is trial 0 with value: 4.626589429747335.
[I 2020-09-11 23:48:55,920] Trial 2 finished with value: 9.103749942047207 and parameters: {'lambda_l1': 6.705360070204016e-05, 'lambda_l2': 2.5513933136096187, 'num_leaves': 247, 'feature_fraction': 0.6436101535620113, 'bagging_fraction': 0.9349967629015011, 'bagging_freq': 7, 'min_child_samples'

[I 2020-09-11 23:48:59,734] Trial 46 finished with value: 4.880577024079102 and parameters: {'lambda_l1': 0.20296612384136054, 'lambda_l2': 2.4300450861481808e-08, 'num_leaves': 159, 'feature_fraction': 0.5223281530967211, 'bagging_fraction': 0.9106347854435476, 'bagging_freq': 7, 'min_child_samples': 5}. Best is trial 17 with value: 4.588108407936272.
[I 2020-09-11 23:48:59,889] Trial 47 finished with value: 5.241452229869473 and parameters: {'lambda_l1': 0.008970942115855954, 'lambda_l2': 1.4844200507782514e-06, 'num_leaves': 164, 'feature_fraction': 0.5109111643384825, 'bagging_fraction': 0.9182930209844973, 'bagging_freq': 7, 'min_child_samples': 5}. Best is trial 17 with value: 4.588108407936272.
[I 2020-09-11 23:48:59,994] Trial 48 finished with value: 5.638080239040954 and parameters: {'lambda_l1': 0.19314790341928698, 'lambda_l2': 2.975289649068602e-08, 'num_leaves': 123, 'feature_fraction': 0.6716253660622774, 'bagging_fraction': 0.8831845599384652, 'bagging_freq': 7, 'min_chi

[I 2020-09-11 23:49:05,238] Trial 92 finished with value: 5.750585644654092 and parameters: {'lambda_l1': 0.9549367600073425, 'lambda_l2': 2.5440000625276737e-06, 'num_leaves': 78, 'feature_fraction': 0.6515426259030632, 'bagging_fraction': 0.7906334004705343, 'bagging_freq': 6, 'min_child_samples': 11}. Best is trial 66 with value: 4.45396954052958.
[I 2020-09-11 23:49:05,397] Trial 93 finished with value: 4.791561615198894 and parameters: {'lambda_l1': 0.3803244317863727, 'lambda_l2': 1.0052705260660035e-06, 'num_leaves': 170, 'feature_fraction': 0.6697613927509591, 'bagging_fraction': 0.8968964421573042, 'bagging_freq': 6, 'min_child_samples': 5}. Best is trial 66 with value: 4.45396954052958.
[I 2020-09-11 23:49:05,509] Trial 94 finished with value: 5.166073628506212 and parameters: {'lambda_l1': 0.03449666979282431, 'lambda_l2': 7.652683892822765, 'num_leaves': 102, 'feature_fraction': 0.4333000776543151, 'bagging_fraction': 0.8510597314855767, 'bagging_freq': 6, 'min_child_sample

[I 2020-09-11 23:49:13,337] Trial 138 finished with value: 5.067418551597785 and parameters: {'lambda_l1': 1.8550743839497068, 'lambda_l2': 4.897203733639438e-07, 'num_leaves': 187, 'feature_fraction': 0.6680351749073645, 'bagging_fraction': 0.9742065013213229, 'bagging_freq': 6, 'min_child_samples': 5}. Best is trial 115 with value: 4.377674787761366.
[I 2020-09-11 23:49:13,402] Trial 139 finished with value: 9.802123949695398 and parameters: {'lambda_l1': 0.2944405415317633, 'lambda_l2': 4.190374852180407e-08, 'num_leaves': 116, 'feature_fraction': 0.7183131273803576, 'bagging_fraction': 0.6486687025912077, 'bagging_freq': 6, 'min_child_samples': 75}. Best is trial 115 with value: 4.377674787761366.
[I 2020-09-11 23:49:13,524] Trial 140 finished with value: 4.916553589152716 and parameters: {'lambda_l1': 0.0752655739455709, 'lambda_l2': 4.2149760424252635e-06, 'num_leaves': 92, 'feature_fraction': 0.40150071874899207, 'bagging_fraction': 0.8244238986110872, 'bagging_freq': 7, 'min_ch

[I 2020-09-11 23:49:19,522] Trial 184 finished with value: 4.628928591628641 and parameters: {'lambda_l1': 0.17568705050756567, 'lambda_l2': 3.466329030915132e-06, 'num_leaves': 78, 'feature_fraction': 0.44437796602587615, 'bagging_fraction': 0.9658926969329701, 'bagging_freq': 6, 'min_child_samples': 9}. Best is trial 115 with value: 4.377674787761366.
[I 2020-09-11 23:49:19,627] Trial 185 finished with value: 5.109261826415189 and parameters: {'lambda_l1': 0.15731778699268978, 'lambda_l2': 2.5226599269308017e-06, 'num_leaves': 79, 'feature_fraction': 0.43274303517441876, 'bagging_fraction': 0.9528757691738018, 'bagging_freq': 6, 'min_child_samples': 11}. Best is trial 115 with value: 4.377674787761366.
[I 2020-09-11 23:49:19,765] Trial 186 finished with value: 4.574735361216138 and parameters: {'lambda_l1': 0.048649806864533005, 'lambda_l2': 3.839939394512927e-06, 'num_leaves': 89, 'feature_fraction': 0.48035764539794135, 'bagging_fraction': 0.9306430342581176, 'bagging_freq': 6, 'mi

...

[I 2020-09-11 23:51:05,052] Trial 826 finished with value: 4.886398204719221 and parameters: {'lambda_l1': 0.7017043724768488, 'lambda_l2': 0.9299936219556714, 'num_leaves': 181, 'feature_fraction': 0.41055371634649784, 'bagging_fraction': 0.9291497519707247, 'bagging_freq': 4, 'min_child_samples': 13}. Best is trial 236 with value: 4.303493112515115.
[I 2020-09-11 23:51:05,201] Trial 827 finished with value: 4.686080278832293 and parameters: {'lambda_l1': 0.9995201450924871, 'lambda_l2': 2.4714855378539684e-06, 'num_leaves': 189, 'feature_fraction': 0.45156776733358833, 'bagging_fraction': 0.8978080877584933, 'bagging_freq': 4, 'min_child_samples': 8}. Best is trial 236 with value: 4.303493112515115.
[I 2020-09-11 23:51:05,354] Trial 828 finished with value: 4.999700802542964 and parameters: {'lambda_l1': 4.028171512152192, 'lambda_l2': 3.1535635941108024e-05, 'num_leaves': 45, 'feature_fraction': 0.574327952817752, 'bagging_fraction': 0.9714338774310614, 'bagging_freq': 2, 'min_child

[I 2020-09-11 23:51:12,466] Trial 871 finished with value: 4.94937480922367 and parameters: {'lambda_l1': 0.13867131043698339, 'lambda_l2': 2.029916268809246e-08, 'num_leaves': 209, 'feature_fraction': 0.4864108086642498, 'bagging_fraction': 0.9216386773427774, 'bagging_freq': 4, 'min_child_samples': 11}. Best is trial 859 with value: 4.230308136623835.
[I 2020-09-11 23:51:12,630] Trial 872 finished with value: 4.773215340256182 and parameters: {'lambda_l1': 0.376818746145699, 'lambda_l2': 2.1231822405373985e-08, 'num_leaves': 211, 'feature_fraction': 0.46248391239397985, 'bagging_fraction': 0.8992037814800335, 'bagging_freq': 6, 'min_child_samples': 7}. Best is trial 859 with value: 4.230308136623835.
[I 2020-09-11 23:51:12,786] Trial 873 finished with value: 4.754372591133819 and parameters: {'lambda_l1': 0.3258148818395084, 'lambda_l2': 1.0003536325928166e-08, 'num_leaves': 210, 'feature_fraction': 0.45261166294402194, 'bagging_fraction': 0.923156546517681, 'bagging_freq': 4, 'min_c

[I 2020-09-11 23:51:20,369] Trial 917 finished with value: 4.662032005498488 and parameters: {'lambda_l1': 0.31624374933270677, 'lambda_l2': 3.6587924797977774e-08, 'num_leaves': 211, 'feature_fraction': 0.45304253921646853, 'bagging_fraction': 0.9427782530715358, 'bagging_freq': 3, 'min_child_samples': 10}. Best is trial 859 with value: 4.230308136623835.
[I 2020-09-11 23:51:20,557] Trial 918 finished with value: 6.19427096575617 and parameters: {'lambda_l1': 0.6518083805920716, 'lambda_l2': 6.155018867767552e-07, 'num_leaves': 79, 'feature_fraction': 0.9790272060152669, 'bagging_fraction': 0.9166118533754072, 'bagging_freq': 4, 'min_child_samples': 7}. Best is trial 859 with value: 4.230308136623835.
[I 2020-09-11 23:51:20,720] Trial 919 finished with value: 4.720704257099329 and parameters: {'lambda_l1': 0.1816840965002783, 'lambda_l2': 1.0347415174583134e-08, 'num_leaves': 214, 'feature_fraction': 0.4658413471133667, 'bagging_fraction': 0.9054699204071269, 'bagging_freq': 6, 'min_c

[I 2020-09-11 23:51:28,091] Trial 961 finished with value: 4.596482717089985 and parameters: {'lambda_l1': 0.27654019183055484, 'lambda_l2': 0.0026460446060265533, 'num_leaves': 231, 'feature_fraction': 0.4436125672468457, 'bagging_fraction': 0.9023157013484645, 'bagging_freq': 4, 'min_child_samples': 7}. Best is trial 859 with value: 4.230308136623835.
[I 2020-09-11 23:51:28,306] Trial 962 finished with value: 4.488069461656881 and parameters: {'lambda_l1': 0.003313097574560688, 'lambda_l2': 2.037767439866808e-06, 'num_leaves': 228, 'feature_fraction': 0.42869831668955816, 'bagging_fraction': 0.9479286765152788, 'bagging_freq': 4, 'min_child_samples': 5}. Best is trial 859 with value: 4.230308136623835.
[I 2020-09-11 23:51:28,478] Trial 963 finished with value: 5.750275563292553 and parameters: {'lambda_l1': 0.6660382514909368, 'lambda_l2': 0.9659323287537217, 'num_leaves': 240, 'feature_fraction': 0.47416767637171797, 'bagging_fraction': 0.8005513222002848, 'bagging_freq': 6, 'min_ch

Number of finished trials: 1000
Best trial:
  Value: 4.230308136623835
  Params: 
    lambda_l1: 0.18488938526436452
    lambda_l2: 1.2464560862630119e-08
    num_leaves: 209
    feature_fraction: 0.45453622320846115
    bagging_fraction: 0.9283200887416692
    bagging_freq: 4
    min_child_samples: 5
